In [ ]:
!nvidia-smi

Mon Jun 28 18:15:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorflow as tf
tf.__version__

'2.5.0'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/train',
                                                 target_size=(150,150),
                                                 batch_size=16,
                                                 class_mode='categorical',)

validation_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/test',
                                                 target_size=(150,150),
                                                 batch_size=16,
                                                 class_mode='categorical')

num_classes = len(train_generator.class_indices)

Found 19059 images belonging to 3 classes.
Found 19800 images belonging to 3 classes.
3


In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras import optimizers
%matplotlib inline
import matplotlib.pyplot as plt

In [1]:
model = Sequential() 
model.add(Flatten(input_shape=(150, 150, 3))) 
model.add(Dense(100, activation=keras.layers.LeakyReLU(alpha=0.3))) 
model.add(Dropout(0.5)) 
model.add(Dense(50, activation=keras.layers.LeakyReLU(alpha=0.3))) 
model.add(Dropout(0.3)) model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), 
              metrics=['acc']) 

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        1

In [ ]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // 16,
        epochs=15,
        validation_data=validation_generator,
        validation_steps=800 // 16)


In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // 16,
        epochs=15,
        validation_data=validation_generator,
        validation_steps=800 // 16)
model.save_weights('first_try.h5')  